In [2]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

2025-11-19 00:27:08.832118: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-19 00:27:08.897769: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-19 00:27:17.848245: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
# 사용자 설정
train_dir = '/workspace/user4/cropped/train'
val_dir = '/workspace/user4/cropped/val'

target_size = (224, 224)
batch_size = 32
seed = 42


In [4]:
# Data generators: EfficientNet 전처리 사용 (preprocess_input)
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # ImageNet 기반 모델용 전처리
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

In [5]:
# 제너레이터: 이진 분류이므로 class_mode='binary'
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    seed=seed
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False,
    seed=seed
)

print("Class indices:", train_generator.class_indices)
if 'happy' in train_generator.class_indices:
    print("'happy' class index:", train_generator.class_indices['happy'])

Found 9550 images belonging to 7 classes.


Found 1151 images belonging to 6 classes.
Class indices: {'anger': 0, 'happy': 1, 'panic': 2, 'sadness': 3, 'train_happy_invalid': 4, 'train_panic_invalid': 5, 'train_sadness_invalid': 6}
'happy' class index: 1


In [7]:
# Base model 로드
base_model = EfficientNetB0(weights='imagenet', include_top=False,
                           input_shape=(target_size[0], target_size[1], 3))

# top 추가
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # 이진 분류

model = Model(inputs=base_model.input, outputs=predictions)

# base freeze
for layer in base_model.layers:
    layer.trainable = False

In [8]:
# 컴파일 (이진 교차엔트로피)
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

model.summary()

# callbacks
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_auc', mode='max',
                             save_best_only=True, verbose=1)
earlystop = EarlyStopping(monitor='val_auc', mode='max', patience=6,
                          restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3,
                              min_lr=1e-6, verbose=1)

# 안전한 step 계산
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,213,668 (16.07 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [9]:
# 학습: 
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,    
)


Epoch 1/10


2025-11-19 00:31:37.606422: I external/local_xla/xla/service/service.cc:163] XLA service 0x7fce3c013780 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-19 00:31:37.606460: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2025-11-19 00:31:38.004197: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-19 00:31:39.817189: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91501
2025-11-19 00:31:40.655406: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-19 00:31:42.196002: I external/local

269/299 ━━━━━━━━━━━━━━━━━━━━ 18s 615ms/step - accuracy: 0.5437 - auc: 0.5031 - loss: -56.4799

2025-11-19 00:34:46.968235: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-19 00:34:47.093691: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-19 00:34:47.463913: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-19 00:34:47.591116: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-19 00:34:48.089792: E external/local_xla/xla/stream_

299/299 ━━━━━━━━━━━━━━━━━━━━ 0s 653ms/step - accuracy: 0.5442 - auc: 0.5028 - loss: -69.7251

2025-11-19 00:35:34.568671: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-19 00:35:34.695564: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-19 00:35:35.099962: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-19 00:35:35.227009: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-11-19 00:35:35.745454: E external/local_xla/xla/stream_

299/299 ━━━━━━━━━━━━━━━━━━━━ 249s 749ms/step - accuracy: 0.5488 - auc: 0.5001 - loss: -210.3504 - val_accuracy: 0.2598 - val_auc: 0.5000 - val_loss: -789.2018
Epoch 2/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 164s 549ms/step - accuracy: 0.5492 - auc: 0.4999 - loss: -1453.0437 - val_accuracy: 0.2598 - val_auc: 0.5000 - val_loss: -3140.4431
Epoch 3/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 167s 558ms/step - accuracy: 0.5492 - auc: 0.4999 - loss: -3917.7893 - val_accuracy: 0.2598 - val_auc: 0.5000 - val_loss: -7021.9458
Epoch 4/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 165s 553ms/step - accuracy: 0.5492 - auc: 0.4999 - loss: -7543.1924 - val_accuracy: 0.2598 - val_auc: 0.5000 - val_loss: -12293.7988
Epoch 5/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 166s 556ms/step - accuracy: 0.5492 - auc: 0.4999 - loss: -12122.6777 - val_accuracy: 0.2598 - val_auc: 0.5000 - val_loss: -18719.4258
Epoch 6/10
299/299 ━━━━━━━━━━━━━━━━━━━━ 166s 555ms/step - accuracy: 0.5492 - auc: 0.4999 - loss: -17640.7930 - val_accuracy: 0.2598 - val_auc: 0.5000 

In [10]:
import pandas as pd

# Extract metrics from the history object for the last epoch
train_loss = history.history['loss'][-1]
train_accuracy = history.history['accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_accuracy'][-1]

# Create a DataFrame to display these metrics
performance_df = pd.DataFrame({
    'Metric': ['Loss', 'Accuracy'],
    'Train': [train_loss, train_accuracy],
    'Validation': [val_loss, val_accuracy]
})

print("\nModel Performance (Last Epoch):")
print(performance_df.to_string(index=False))


Model Performance (Last Epoch):
  Metric         Train    Validation
    Loss -47419.335938 -65964.929688
Accuracy      0.549215      0.259774
